In [3]:
# %%
import ee
import geemap
import geopandas as gpd


# Authenticate once (browser popup). After this, ee.Initialize() works.
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

print("✅ Earth Engine initialized")


EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access

In [ ]:
# %%
# Load AOI from your repo GeoJSON and convert to EE geometry
AOI_GEOJSON = r"C:\GIS\my_water_projects\openwater-shrinking-lake-monitor\data\external\aoi.geojson"


gdf = gpd.read_file(AOI_GEOJSON)
print("GeoJSON CRS:", gdf.crs)

aoi_fc = geemap.geojson_to_ee(AOI_GEOJSON)
aoi = aoi_fc.geometry()

print("✅ AOI loaded into Earth Engine")
